# 1. Multiple-input model

This notebook provides an example of building a neural net with multiple inputs. Specifically, we are training a digit recognizer with half of the image as input 1, and the other half as input 2.

**Warning**: In practice you probably don't want to split an image into two halves. Here we just illustrate how to build a neural net with multiple inputs.

---

We first load the MNIST data

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


We preprocess the image as previously

In [2]:
from utils import preprocess

train_images = preprocess(train_images)
test_images = preprocess(test_images)

We also need to one-hot encode the labels

In [3]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Then we need to split each image into two halves: one half to be used as input stream 1, while the other half to be used as input stream 2

In [4]:
mid = 14 # the image is 28 x 28
train_1 = train_images[:, :mid, :, :]
train_2 = train_images[:, mid:, :, :]

We can construct our neural networks now

In [5]:
from keras import layers
from keras import Input
from keras.models import Model

Construct stream 1

In [6]:
input_1 = Input(shape=(mid, 28, 1))
stream_1 = layers.Conv2D(32, (3, 3), activation='relu')(input_1)
stream_1 = layers.MaxPool2D(2, 2)(stream_1)
stream_1 = layers.Conv2D(64, (3, 3), activation='relu')(stream_1)
stream_1 = layers.MaxPool2D(2, 2)(stream_1)

Construct stream 2

In [7]:
input_2 = Input(shape=(mid, 28, 1))
stream_2 = layers.Conv2D(32, (3, 3), activation='relu')(input_2)
stream_2 = layers.MaxPool2D(2, 2)(stream_2)
stream_2 = layers.Conv2D(64, (3, 3), activation='relu')(stream_2)
stream_2 = layers.MaxPool2D(2, 2)(stream_2)

Concatenate the two streams

In [8]:
concat = layers.concatenate([stream_1, stream_2])

Flatten and add dense layers on top

In [9]:
output = layers.Flatten()(concat)
output = layers.Dense(64, activation='relu')(output)
output = layers.Dense(10, activation='softmax')(output)

Instantiate the model with 2 inputs and 1 output, and compile the model

In [10]:
model = Model([input_1, input_2], output)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train model

In [11]:
model.fit([train_1, train_2], train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 42s - loss: 0.1991 - acc: 0.9389    
Epoch 2/5
60000/60000 [==============================] - 40s - loss: 0.0649 - acc: 0.9796    
Epoch 3/5
60000/60000 [==============================] - 38s - loss: 0.0465 - acc: 0.9856    
Epoch 4/5
60000/60000 [==============================] - 42s - loss: 0.0357 - acc: 0.9892    
Epoch 5/5
60000/60000 [==============================] - 41s - loss: 0.0289 - acc: 0.9910    


And test the model on unseen data

In [12]:
test_1 = test_images[:, :mid, :, :]
test_2 = test_images[:, mid:, :, :]

test_loss, test_acc = model.evaluate([test_1, test_2], test_labels)

10000/10000 [==============================] - 2s     


In [13]:
test_acc

0.98770000000000002

Looks like the accuracy is comparable to simple neural net model, but the training time is 20% less.

Again, don't take this result seriously - just want to illustrate how to build a multiple-input model.